## IMPORT LIB

In [1]:
import pandas as pd 
import numpy as np
import math as math

In [2]:
#  datasetInput = ratingData.pivot(index = 'user_id', columns = 'item_id', values='rating')
#     # fill NaN with 0
#     insertDataNew = datasetInput.fillna(0)

## TOY DATA

In [3]:
toyDataMatriks = [
    [3,	0,	0,	0,	0,	0,	2],
    [1,	2,	4,	2,	2,	0,	0],
    [0,	4,	1,	0,	0,	2,	4],
    [0,	0,	0,	0,	3,	2,	4]
]

In [4]:
# toydata to dataframe

dfToyData = pd.DataFrame(toyDataMatriks, columns=['item-1', 'item-2', 'item-3', 'item-4','item-5','item-6', 'item-7'], index=['user-1', 'user-2', 'user-3', 'user-4'])
dfToyData


,item-1,item-2,item-3,item-4,item-5,item-6,item-7
user-1,3,0,0,0,0,0,2
user-2,1,2,4,2,2,0,0
user-3,0,4,1,0,0,2,4
user-4,0,0,0,0,3,2,4


In [5]:
dfToyData2 = pd.DataFrame(
    toyDataMatriks,
    columns=[1, 2, 3, 4, 5, 6, 7],  # kolom menggunakan integer
    index=[1, 2, 3, 4]  # index juga menggunakan integer
)

dfToyData2

,1,2,3,4,5,6,7
1,3,0,0,0,0,0,2
2,1,2,4,2,2,0,0
3,0,4,1,0,0,2,4
4,0,0,0,0,3,2,4


## MEAN

In [6]:
def meanJaccard(rating):
    pembilang = rating.sum(axis=1)
    print('pembilang', pembilang)
    penyebut = np.count_nonzero(rating, axis=1)
    # menghindari pembagian dengan 0
    rumusMean = pembilang / penyebut
    calCulatedMean  = np.where(penyebut != 0, rumusMean, 0)
    # menghitung mean
    meanRatingUser = pd.DataFrame(calCulatedMean, index=rating.index, columns=['meanRating'])
    return meanRatingUser

In [7]:
cal_MeanUser = meanJaccard(dfToyData)
cal_MeanUser

pembilang user-1     5
user-2    11
user-3    11
user-4     9
dtype: int64


,meanRating
user-1,2.50
user-2,2.20
user-3,2.75
user-4,3.00


In [8]:
cal_MeanItem = meanJaccard(dfToyData.T)
cal_MeanItem

pembilang item-1     4
item-2     6
item-3     5
item-4     2
item-5     5
item-6     4
item-7    10
dtype: int64


,meanRating
item-1,2.000000
item-2,3.000000
item-3,2.500000
item-4,2.000000
item-5,2.500000
item-6,2.000000
item-7,3.333333


In [9]:
# # tranpose

# cal_MeanItemTranpose = cal_MeanItem.T
# cal_MeanItemTranpose

## MEAN CENTERED

In [10]:
# Creating a 1D NumPy array 
arr = np.array([1, 2, 3, 4, 5, 6])

# Reshaping the array into a 2D array
# '-1' allows to calculate the number of rows based on the total number of elements
reshaped_arr = np.reshape(arr, (-1, 1))  
print(reshaped_arr)

[[1]
 [2]
 [3]
 [4]
 [5]
 [6]]


In [11]:
def meanCenterd(rating, meanRating):
    npRatingMatriks = np.array(rating)
    npMeanRatingUser = np.array(meanRating)
    # !0
    rumusMeanCenterd = npRatingMatriks - npMeanRatingUser
    meanCenteredUser = np.where(npRatingMatriks!=0, rumusMeanCenterd,0)
    dfMeanCenteredUser = pd.DataFrame(meanCenteredUser, index=rating.index, columns=rating.columns)

    return dfMeanCenteredUser

In [12]:
cal_MeanCenteredUser = meanCenterd(dfToyData, cal_MeanUser)
cal_MeanCenteredUser

,item-1,item-2,item-3,item-4,item-5,item-6,item-7
user-1,0.5,0.00,0.00,0.0,0.0,0.00,-0.50
user-2,-1.2,-0.20,1.80,-0.2,-0.2,0.00,0.00
user-3,0.0,1.25,-1.75,0.0,0.0,-0.75,1.25
user-4,0.0,0.00,0.00,0.0,0.0,-1.00,1.00


In [13]:
cal_MeanCenteredItem = meanCenterd(dfToyData.T, cal_MeanItem)
cal_MeanCenteredItem


,user-1,user-2,user-3,user-4
item-1,1.000000,-1.0,0.000000,0.000000
item-2,0.000000,-1.0,1.000000,0.000000
item-3,0.000000,1.5,-1.500000,0.000000
item-4,0.000000,0.0,0.000000,0.000000
item-5,0.000000,-0.5,0.000000,0.500000
item-6,0.000000,0.0,0.000000,0.000000
item-7,-1.333333,0.0,0.666667,0.666667


In [14]:
cal_MeanCenteredItemTranpose = cal_MeanCenteredItem.T
cal_MeanCenteredItemTranpose

,item-1,item-2,item-3,item-4,item-5,item-6,item-7
user-1,1.0,0.0,0.0,0.0,0.0,0.0,-1.333333
user-2,-1.0,-1.0,1.5,0.0,-0.5,0.0,0.000000
user-3,0.0,1.0,-1.5,0.0,0.0,0.0,0.666667
user-4,0.0,0.0,0.0,0.0,0.5,0.0,0.666667


## USER-BASED

In [15]:
def simJaccardDuaUser(rating, penggunaU, penggunaV):
    # Dapatkan item yang dirating (tidak sama dengan 0)
    itemRated1 = set(np.where(rating.loc[penggunaU, :] != 0)[0])
    print('itemRated1', itemRated1)
    itemRated2 = set(np.where(rating.loc[penggunaV, :] != 0)[0])
    print('itemRated2', itemRated2)
    itemRatedPenggunaU = set(rating.columns[rating.loc[penggunaU, : ] != 0])
    print('itemRatedPenggunaU', itemRatedPenggunaU)
    itemRatedPenggunaV = set(rating.columns[rating.loc[penggunaV, : ] != 0])
    print('itemRatedPenggunaV', itemRatedPenggunaV)
    itemRatedPenggunaU2 = set(rating.columns[rating.iloc[penggunaU, : ] != 0])
    print('itemRatedPenggunaU2', itemRatedPenggunaU2)
    itemRatedPenggunaV2 = set(rating.columns[rating.iloc[penggunaV, : ] != 0])
    print('itemRatedPenggunaV2', itemRatedPenggunaV2)
    # itemNoRated1 = set(np.where(RatingMatriks.loc[index1, :] == 0)[0])
    #print('item no rated1', itemNoRated1)
    # Hitung Jaccard similarity
    intersection2 = len(itemRated1.intersection(itemRated2))
    union2 = len(itemRated1.union(itemRated2))
    print('intersection', intersection2)
    intersection = len(itemRatedPenggunaU.intersection(itemRatedPenggunaV))
    print('intersection', intersection)
    union = len(itemRatedPenggunaU.union(itemRatedPenggunaV))
    print('union', union)
    rumus = intersection / union
    rumus2 = intersection2 / union2
    print(f'jac,, {rumus} ')
    print(f'jac2,, {rumus2} ')

    if union != 0:
        return intersection2 / union
    else:
        return 0


In [16]:
cal_SimDuaUser = simJaccardDuaUser(dfToyData2, 1, 3)
print(f"Jaccard Similarity between user-1 and user-3 is {cal_SimDuaUser}")

itemRated1 {0, 6}
itemRated2 {1, 2, 5, 6}
itemRatedPenggunaU {1, 7}
itemRatedPenggunaV {2, 3, 6, 7}
itemRatedPenggunaU2 {1, 2, 3, 4, 5}
itemRatedPenggunaV2 {5, 6, 7}
intersection 1
intersection 1
union 5
jac,, 0.2 
jac2,, 0.2 
Jaccard Similarity between user-1 and user-3 is 0.2


In [17]:
def simJaccardDuaUser(rating, penggunaU, penggunaV):
    # Dapatkan item yang dirating (tidak sama dengan 0)
    itemRatedPenggunaU = set(rating.columns[rating.iloc[penggunaU, : ] != 0])
    print('itemRatedPenggunaU', itemRatedPenggunaU)
    itemRatedPenggunaV = set(rating.columns[rating.iloc[penggunaV, : ] != 0])
    print('itemRatedPenggunaV', itemRatedPenggunaV)
    # Hitung Jaccard similarity
    intersection = len(itemRatedPenggunaU.intersection(itemRatedPenggunaV))
    union = len(itemRatedPenggunaU.union(itemRatedPenggunaV))

    if union != 0:
        return intersection / union
    else:
        return 0


In [18]:
calDuaUser = simJaccardDuaUser(dfToyData2, 0, 1)
print(f"Jaccard Similarity between user-1 and user-2 is {calDuaUser}")

itemRatedPenggunaU {1, 7}
itemRatedPenggunaV {1, 2, 3, 4, 5}
Jaccard Similarity between user-1 and user-2 is 0.16666666666666666


## ITEM BASED

In [19]:
def simJaccardDuaItem(rating, itemI, itemJ):
    # mendapatkan user yang sudah memberikan rating pada item I
    itemRatedItemI = set(rating.index[rating.iloc[:, itemI] != 0])
    # user yang sudah memberikan raitng pada item J
    itemRatedItemJ = set(rating.index[rating.iloc[:, itemJ] != 0])


    # Hitung Jaccard similarity
    intersection = len(itemRatedItemI.intersection(itemRatedItemJ))
    union = len(itemRatedItemI.union(itemRatedItemJ))

    rumusJaccardItem = intersection/union
    if intersection != 0:
        return rumusJaccardItem
    else:
        return 0


In [20]:
simJaccardDuaItemW = simJaccardDuaItem(dfToyData, 0, 1)
print(f"Jaccard Similarity between item-1 and item-2 is {simJaccardDuaItemW}")

Jaccard Similarity between item-1 and item-2 is 0.3333333333333333


## ALL SIMILARITY

In [21]:
def similarityAllUserItem(rating, jenis="user"):
    if jenis == "user" : 
        jumlahUser = rating.shape[0]
        matriksSimilarity = np.zeros((jumlahUser, jumlahUser))
        np.fill_diagonal(matriksSimilarity, 1)
        for i in range (jumlahUser):
            for j in range(i+1, jumlahUser):
                rumusSimJaccardUser = simJaccardDuaUser(rating, i, j)
                matriksSimilarity[i][j] = rumusSimJaccardUser
                matriksSimilarity[j][i] = rumusSimJaccardUser
        simMatriksDF = pd.DataFrame(matriksSimilarity, index=rating.index, columns=rating.index)
    elif jenis == "item" : 
        jumlahItem = rating.shape[1]
        matriksSimilarity = np.zeros((jumlahItem, jumlahItem))
        np.fill_diagonal(matriksSimilarity, 1)
        for i in range (jumlahItem):
            for j in range(i+1, jumlahItem):
                rumusSimJaccardItem = simJaccardDuaItem(rating, i, j)
                matriksSimilarity[i][j] = rumusSimJaccardItem
                matriksSimilarity[j][i] = rumusSimJaccardItem
        simMatriksDF = pd.DataFrame(matriksSimilarity, index=rating.columns, columns=rating.columns)
    return simMatriksDF

In [22]:
cal_SimUserJaccard = similarityAllUserItem(dfToyData, "user")
cal_SimUserJaccard

itemRatedPenggunaU {'item-1', 'item-7'}
itemRatedPenggunaV {'item-1', 'item-2', 'item-5', 'item-4', 'item-3'}
itemRatedPenggunaU {'item-1', 'item-7'}
itemRatedPenggunaV {'item-3', 'item-2', 'item-6', 'item-7'}
itemRatedPenggunaU {'item-1', 'item-7'}
itemRatedPenggunaV {'item-7', 'item-6', 'item-5'}
itemRatedPenggunaU {'item-1', 'item-2', 'item-5', 'item-4', 'item-3'}
itemRatedPenggunaV {'item-3', 'item-2', 'item-6', 'item-7'}
itemRatedPenggunaU {'item-1', 'item-2', 'item-5', 'item-4', 'item-3'}
itemRatedPenggunaV {'item-7', 'item-6', 'item-5'}
itemRatedPenggunaU {'item-3', 'item-2', 'item-6', 'item-7'}
itemRatedPenggunaV {'item-7', 'item-6', 'item-5'}


,user-1,user-2,user-3,user-4
user-1,1.000000,0.166667,0.200000,0.250000
user-2,0.166667,1.000000,0.285714,0.142857
user-3,0.200000,0.285714,1.000000,0.400000
user-4,0.250000,0.142857,0.400000,1.000000


In [23]:
cal_SimItemJaccard = similarityAllUserItem(dfToyData, "item")
cal_SimItemJaccard

,item-1,item-2,item-3,item-4,item-5,item-6,item-7
item-1,1.000000,0.333333,0.333333,0.5,0.333333,0.000000,0.250000
item-2,0.333333,1.000000,1.000000,0.5,0.333333,0.333333,0.250000
item-3,0.333333,1.000000,1.000000,0.5,0.333333,0.333333,0.250000
item-4,0.500000,0.500000,0.500000,1.0,0.500000,0.000000,0.000000
item-5,0.333333,0.333333,0.333333,0.5,1.000000,0.333333,0.250000
item-6,0.000000,0.333333,0.333333,0.0,0.333333,1.000000,0.666667
item-7,0.250000,0.250000,0.250000,0.0,0.250000,0.666667,1.000000


## PREDIKSI 


### TOP-K

In [74]:
def TetanggaK(target_user, target_item, matriks_rating, similarity, k, jenis="user-based"):
    rating_matrix = matriks_rating.to_numpy()  # Convert ke NumPy untuk efisiensi
    if jenis == "user-based":
        # mendapatkan rating dari user yang ditargetkan
        item_ratings = rating_matrix[:, target_item]
        # mencari item yang sudah di rating oleh user
        item_rated_by = np.flatnonzero(item_ratings)
        # print(f"item rated by {len(item_rated_by)}")
        item_rated_by = item_rated_by[item_rated_by != target_user]
        # ambil valaue similarity dari user yang ditargetkan ke item yang sudah di rating oleh user
        sim_scores = similarity.values[target_user, item_rated_by]
        indices = item_rated_by
    else:  # item-based
        user_ratings = rating_matrix[target_user, :]
        # print(f'user rating {len(user_ratings)}')
        items_rated = np.flatnonzero(user_ratings)
        items_rated = items_rated[items_rated != target_item]

        sim_scores = similarity.values[items_rated, target_item]
        indices = items_rated
    #diurutka nilai terbesar ke terkecil
    sorted_idx = np.argsort(-sim_scores)[:k]
    # simScores = sim_scores
    print(f"sim scores {sim_scores}")
    print(f"sorted idx {sorted_idx}")
    # maping tetang ulang berdasarkan index asli 
    top_neighbors = [indices[i] for i in sorted_idx]
    print(f"top neighbors {top_neighbors}")
    print(f"sim scores2 {sim_scores[sorted_idx]}")
    return top_neighbors


In [75]:
calTetanggaK = TetanggaK(0, 1, dfToyData, cal_SimItemJaccard, 5, "item-based")
print(f"tetangga K {calTetanggaK}")

sim scores [0.33333333 0.25      ]
sorted idx [0 1]
top neighbors [0, 6]
sim scores2 [0.33333333 0.25      ]
tetangga K [0, 6]


In [ ]:
# def TetanggaK2(target_user, target_item, matriks_rating, similarity, k, jenis="user-based"):
#     # jika jenis user-based
#     if jenis == "user-based":
#         # daftar user lain yang sudah memberi rating pada item target
#         # memilih user yang tidak sama dengan target user
#         # dan hanya memilih user yang telah memberikan rating pada item target, yang lebih dari 0
#         filter_indices = [
#             u for u in matriks_rating.index
#             if u != target_user and matriks_rating.loc[u, target_item] > 0
#         ]
#         # Ambil nilai similarity terhadap user target yang ada di filter indices, baris target user dan kolom user pada filter, array
#         sim_scores = similarity.loc[target_user, filter_indices].values
#     # jika jenis item-based
#     elif jenis == "item-based":
#         # Ambil item-item lain yang pernah diberi rating oleh user target
#         # memilih item yang tidak sama dengan target item
#         # dan hanya memilih item yang telah memberikan rating pada user target, yang lebih dari 0
#         filter_indices = [
#             i for i in matriks_rating.columns
#             if i != target_item and matriks_rating.loc[target_user, i] > 0
#         ]
#         # Ambil nilai similarity terhadap item target yang ada di filter indices, baris target item dan kolom item pada filter, array
#         sim_scores = similarity.loc[target_item, filter_indices].values
#     #mengambil index dari banyak k yang paling mirip, paling besar, mengambil k indexs pertama
#     top_k_idx = np.argsort(-sim_scores)[:k]
#     print("top_k_idx", top_k_idx)
#     # mapping ulang sesuai dengan index pada rating matriks
#     top_k_items = [filter_indices[i] for i in top_k_idx]
#     print(sim_scores[top_k_idx])
#     return top_k_items


In [ ]:
# def TetanggaK(target_user, target_item, matriks_rating, similarity, k, jenis="user"):
#     if jenis == "user":
#         # Cari user lain (index) yang memberi rating pada item tertentu
#         #baris
#         filter_indices = [i for i in range(matriks_rating.shape[0])
#                             #tidak dirinya sendiri, cek user i memberi rating pada item target, list,  baris dan kolom
#                             #akan mengembalikan nilai rating yang diberikan oleh user ke-i terhadap item ke-target. yang lebih dari 0
#                             if i != target_user and matriks_rating.iloc[i, target_item] > 0]

#         print(f"Jumlah user: {matriks_rating.shape[0]}")
#         print(f"Target user: {target_user}")
#         print(f"Target item: {target_item}")

        

#         # yang belum di rating
#         filterbELUMRating = [i for i in range(matriks_rating.shape[0])
#                             #tidak dirinya sendiri, cek user i memberi rating pada item target, list
#                             if  matriks_rating.iloc[i, target_item] == 0]
        
#         print(filterbELUMRating)

#         print("filter index", filter_indices)
#         #index, baris, kolom kandidat yang di ambil, array
#         sim_scores = similarity.iloc[target_user, filter_indices].values
#         print("sim", sim_scores)
#     else:
#         # Cari item lain (index) yang pernah dinilai oleh user tertentu
#         #kolom
#         filter_indices = [j for j in range(matriks_rating.shape[1])
#                             #tidak dirinya sendiri, cek user target memberi rating pada item j, list
#                             if j != target_user and matriks_rating.iloc[target_item, j] > 0]
#         #index, kolom, baris kandidat yang di ambil, array
#         sim_scores = similarity.iloc[target_user, filter_indices].values

#     # Urutkan index berdasarkan similarity, dari yang tertinggi ke terendah, index k
#     top_k_idx = np.argsort(-sim_scores)[:k]
#     print("top k", top_k_idx)
#     # list comprehension, list int
#     top_k_indices = [filter_indices[i] for i in top_k_idx]
#     # list int
#     return top_k_indices


In [72]:
def prediksiCF_v2(RatingMatriks, similarityFunction, mean, meanCen, user=2, item=2, k=2, jenis="user-based"):
    # jika jenis user-based
    if jenis == "user-based":
        target_user1 = user
        target_item2 = item
        # ambil mean rating user target
        mean_value = mean.iloc[user]
        # print("mean", mean_value)
    elif jenis == "item-based":
        target_user1 = item
        target_item2 = user
        # ambil mean rating item target
        mean_value = mean.iloc[item]
    # Ambil top-k tetangga
    tetangga = TetanggaK(
        target_user=target_user1,
        target_item=target_item2,
        matriks_rating=RatingMatriks,
        similarity=similarityFunction,
        k=k,
        jenis=jenis
    )
    # print(f"tetangga {len(tetangga)}")
    # menyiapkan pembilang dan penyebut untuk menghitung prediksi rating
    pembilang = 0
    penyebut = 0
    # loop sesuai dengan banyak tetangga
    for tetangga_idx in tetangga:
        if jenis == "user-based":
            # mengambil sim antara user dan tetangga
            sim = similarityFunction.iloc[user, tetangga_idx]
            # print(f"sim {user}-{tetangga_idx} = {sim}")
            mean_cen = meanCen.iloc[tetangga_idx, item]
            # print(f"mean_cen {tetangga_idx}-{item} = {mean_cen}")
        else:  # item-based
            # mengambil sim antara item dan tetangga
            sim = similarityFunction.iloc[tetangga_idx, item]
            # mengambil mean centered rating untuk user target dari tetangga
            mean_cen = meanCen.iloc[user, tetangga_idx]
        pembilang += mean_cen * sim
        # print("pembilang", (pembilang))
        # menghitung penyebut
        penyebut += abs(sim)
        # print("penyebut", penyebut)
    # jika penyebut == 0, maka hitung predikis
    if penyebut != 0:
        prediksi = mean_value + (pembilang / penyebut)
        # print(f"prediks1 {prediksi} = {mean_value} + ({pembilang} / {penyebut})")
    else:
        prediksi = mean_value
    return [item, float(prediksi)]

In [77]:
calPrediksiItem = prediksiCF_v2(dfToyData, cal_SimItemJaccard, cal_MeanItem, cal_MeanCenteredItemTranpose, user=0, item=1, k=10, jenis="item-based")
calPrediksiItem

sim scores [0.33333333 0.33333333 0.5        0.33333333]
sorted idx [2 0 1 3]
top neighbors [3, 1, 2, 4]
sim scores2 [0.5        0.33333333 0.33333333 0.33333333]


[1, 3.0]

In [ ]:
# import numpy as np
# import pandas as pd

# def TetanggaK4(target_user, target_item, rating_matrix_np, similarity_np, k, jenis="user-based"):
#     """
#     Mengembalikan indeks K tetangga terdekat (user atau item) berdasarkan similarity.
#     """
#     rating_matrix = rating_matrix_np
#     if jenis == "user-based":
#         # Ambil semua rating terhadap item target
#         item_ratings = rating_matrix[:, target_item]
#         # User yang pernah memberi rating terhadap item tersebut (kecuali user target)
#         tetangga_idx = np.where((item_ratings != 0) & (np.arange(len(item_ratings)) != target_user))[0]
#         # Ambil skor similarity
#         sim_scores = similarity_np[target_user, tetangga_idx]
#         # valid_idx = tetangga_idx[rating_matrix_np[tetangga_idx, target_item] != 0]
#         # sim_scores = similarity_np[target_user, valid_idx]

#     else:  # item-based         
#         user_ratings = rating_matrix[target_user, :]
#         tetangga_idx = np.where((user_ratings != 0) & (np.arange(len(user_ratings)) != target_item))[0]
#         sim_scores = similarity_np[tetangga_idx, target_item]
#         # valid_idx = tetangga_idx[rating_matrix_np[target_user, tetangga_idx] != 0]
#         # sim_scores = similarity_np[valid_idx, target_item]
#     # Urutkan tetangga berdasarkan similarity tertinggi
#     if len(sim_scores) == 0:
#         return []
#     sorted_idx = np.argsort(-sim_scores)[:k]
#     print(f"sorted idx {sorted_idx}")
#     print(f"sim from soreted {sorted_idx} {sim_scores[sorted_idx]}")
#     print(f"tetangga idx {tetangga_idx[sorted_idx]}")
#     return tetangga_idx[sorted_idx]



In [ ]:
# def prediksiCF_v2_optimized(RatingMatriks, similarityFunction, mean, meanCen, user, item, k=2, jenis="user-based"):
#     """
#     Menghitung prediksi rating menggunakan Collaborative Filtering berbasis user/item, versi optimal.
#     """

#     # Cache ke bentuk NumPy (lebih cepat)
#     rating_np = RatingMatriks.values
#     similarity_np = similarityFunction.values
#     mean_np = mean.values
#     meanCen_np = meanCen.values

#     # Tetapkan target user dan item sesuai jenis
#     target_user, target_item = (user, item) if jenis == "user-based" else (item, user)
#     mean_value = mean_np[user if jenis == "user-based" else item]

#     # Cari tetangga terdekat
#     tetangga = TetanggaK4(target_user, target_item, rating_np, similarity_np, k, jenis)

#     if len(tetangga) == 0:
#         return float(mean_value)

#     # Vektorisasi similarity dan mean centered rating
#     if jenis == "user-based":
#         sim = similarity_np[user, tetangga]
#         mean_cen = meanCen_np[tetangga, item]
#     else:
#         sim = similarity_np[tetangga, item]
#         mean_cen = meanCen_np[user, tetangga]

#     # Hitung prediksi
#     pembilang = np.dot(mean_cen, sim)
#     penyebut = np.sum(np.abs(sim))

#     prediksi = mean_value + (pembilang / penyebut) if penyebut != 0 else mean_value
#     return float(prediksi)


In [79]:
calPrediksiv2 = prediksiCF_v2_optimized(dfToyData, cal_SimItemJaccard, cal_MeanItem, cal_MeanCenteredItemTranpose, user=0, item=1, k=10, jenis="item-based")
print(f"Prediksi rating untuk user-1 pada item-2: {calPrediksiv2}")

sorted idx [2 0 1 3]
sim from soreted [2 0 1 3] [0.5        0.33333333 0.33333333 0.33333333]
tetangga idx [3 1 2 4]
Prediksi rating untuk user-1 pada item-2: 3.0


### PREDIKSI KESELURUHAN 

In [32]:
def hitungKeseluruhanPrediksi(RatingMatriks, similarityFunction, mean, meanCen, k=2, jenis="userBased"):
    # Membuat DataFrame kosong untuk menyimpan prediksi
    # prediksiMatriks = pd.DataFrame(index=RatingMatriks.index, columns=RatingMatriks.columns)
    prediksiMatriks = RatingMatriks.copy()
    
    # Iterasi untuk setiap user dan item dalam matriks rating
    #baris
    for user in RatingMatriks.index:
        #kolom
        for item in RatingMatriks.columns:
            # Cek apakah rating untuk (user, item) adalah 0 atau tidak ada rating
            if RatingMatriks.loc[user, item] == 0:
                # Jika tidak ada rating, prediksi nilai rating menggunakan Collaborative Filtering
                prediksiMatriks.loc[user, item] = prediksiCF_v2(
                    RatingMatriks,
                    similarityFunction,
                    mean,
                    meanCen,
                    user=user,
                    item=item,
                    k=k,
                    jenis=jenis
                )[1]
            else:
                # Jika sudah ada rating, simpan rating asli dari RatingMatriks
                prediksiMatriks.loc[user, item] = RatingMatriks.loc[user, item]
    return prediksiMatriks


In [33]:
cal_HitungAlUser = hitungKeseluruhanPrediksi(dfToyData, cal_SimUserJaccard, cal_MeanUser, cal_MeanCenteredUser, k=2, jenis="userBased")
cal_HitungAlUser

Index user: 0
Jumlah user: 4
Target user: 0
Target item: 1
[0, 3]
filter index [1, 2]
sim [0.16666667 0.2       ]
top k [1 0]
Nama tetangga: user-3
Nama tetangga: user-2
Index user: 0
Jumlah user: 4
Target user: 0
Target item: 2
[0, 3]
filter index [1, 2]
sim [0.16666667 0.2       ]
top k [1 0]
Nama tetangga: user-3
Nama tetangga: user-2
Index user: 0
Jumlah user: 4
Target user: 0
Target item: 3
[0, 2, 3]
filter index [1]
sim [0.16666667]
top k [0]
Nama tetangga: user-2
Index user: 0
Jumlah user: 4
Target user: 0
Target item: 4
[0, 2]
filter index [1, 3]
sim [0.16666667 0.25      ]
top k [1 0]
Nama tetangga: user-4
Nama tetangga: user-2
Index user: 0
Jumlah user: 4
Target user: 0
Target item: 5
[0, 1]
filter index [2, 3]
sim [0.2  0.25]
top k [1 0]
Nama tetangga: user-4
Nama tetangga: user-3
Index user: 1
Jumlah user: 4
Target user: 1
Target item: 5
[0, 1]
filter index [2, 3]
sim [0.28571429 0.14285714]
top k [0 1]
Nama tetangga: user-3
Nama tetangga: user-4
Index user: 1
Jumlah user: 

,item-1,item-2,item-3,item-4,item-5,item-6,item-7
user-1,3.000000,3.090909,2.363636,2.30,2.420000,1.611111,2.000000
user-2,1.000000,2.000000,4.000000,2.00,2.000000,1.366667,2.805263
user-3,2.250000,4.000000,1.000000,2.55,2.666667,2.000000,4.000000
user-4,2.881818,3.868421,2.184211,2.80,3.000000,2.000000,4.000000


In [34]:
cal__HitungAllItem = hitungKeseluruhanPrediksi(dfToyData, cal_SimItemJaccard, cal_MeanItem, cal_MeanCenteredItemTranpose, k=2, jenis="itemBased")
cal__HitungAllItem

top k [0 1]
top k [0 1]
top k [0 1]
top k [0 1]
top k [1 0]
top k [1 2]
top k [0 1]
top k [0 1]
top k [0 1]
top k [0 1]
top k [0 2]
top k [0 1]
top k [0 1]
top k [0 1]


,item-1,item-2,item-3,item-4,item-5,item-6,item-7
user-1,3.000000,3.00,2.50,3.00,2.50,0.666667,2.000000
user-2,1.000000,2.00,4.00,2.00,2.00,2.250000,2.333333
user-3,1.750000,4.00,1.00,1.75,2.25,2.000000,4.000000
user-4,2.571429,3.25,2.75,2.50,3.00,2.000000,4.000000


## HYBRID

In [35]:
def HybridFiltering(PrediksiUser, PrediksiItem, gamma=0.7):
    # Mengonversi ke numpy array
    # prediksiUser = np.array(PrediksiUser)
    # print("prediksi user", prediksiUser)
    # prediksiItem = np.array(PrediksiItem)
    # print("prediksi item", prediksiItem)
    # Menghitung prediksi hybrid
    rumusHybrid = (gamma * PrediksiUser) + ((1 - gamma) * PrediksiItem)  

    return rumusHybrid


In [36]:
cal_HybridJac = HybridFiltering(cal_HitungAlUser, cal__HitungAllItem, gamma=0.7)
cal_HybridJac

,item-1,item-2,item-3,item-4,item-5,item-6,item-7
user-1,3.000000,3.063636,2.404545,2.51,2.444000,1.327778,2.000000
user-2,1.000000,2.000000,4.000000,2.00,2.000000,1.631667,2.663684
user-3,2.100000,4.000000,1.000000,2.31,2.541667,2.000000,4.000000
user-4,2.788701,3.682895,2.353947,2.71,3.000000,2.000000,4.000000


In [37]:
prediksi_hybrid_user1 = cal_HybridJac[0]
print("Hybrid Prediction for User-1:", prediksi_hybrid_user1)

KeyError: 0

In [ ]:
def TopNRecommendation(rekomendasiHybrid, N=5):
    topN_per_user = np.argsort(-rekomendasiHybrid, axis=1)[:N]+1
    # tolist
    topN_per_user = topN_per_user.tolist()

    return topN_per_user


In [ ]:
calTpn = TopNRecommendation(cal_HybridJac)
calTpn

[[2, 1, 4, 5, 3, 7, 6],
 [3, 7, 2, 4, 5, 6, 1],
 [2, 7, 5, 4, 1, 6, 3],
 [7, 2, 5, 1, 4, 3, 6]]

In [ ]:
def FullRecommendation(prediksi_matrix):
    ranking_per_user = np.argsort(-prediksi_matrix, axis=1)+1
    return ranking_per_user

## TOP N REKOMENDASI V2

In [ ]:
def getTopNRekomendasiV3(prediksiHybrid, ratingMatrix, user_index, N=5):
    #ambil seluruh prediksi user dan rating user. array
    pred_user = prediksiHybrid.iloc[user_index].values
    # print("prediksi user", pred_user)
    rating_user = ratingMatrix.iloc[user_index].values
    # print("rating user", rating_user)
    # Item yang belum dirating (nilai 0 atau NaN)

    if np.issubdtype(rating_user.dtype, np.number):
        belum_dirating_idx = np.where(rating_user == 0)[0]
        # print("belum dirating", belum_dirating_idx)
    else:
        belum_dirating_idx = np.where(pd.isna(rating_user))[0]
        print("belum dirating", belum_dirating_idx)
    # Prediksi hanya untuk item yang belum dirating, array float
    pred_belum_dirating = pred_user[belum_dirating_idx]
    # array numpy, amhil yang belum dirating, ambil topN yang belum dirating
    top_indices = pred_belum_dirating.argsort()[::-1][:N] # membalik urutan, ambil N pertama
    top_item_ids = prediksiHybrid.columns.to_numpy()[belum_dirating_idx][top_indices]
    #array
    return list(top_item_ids)


In [ ]:
topNRekomendasiTopN = getTopNRekomendasiV3(cal_HybridJaccard, dfToyData, 0, N=3)
print(f"Top-N recommendations for user-1: {topNRekomendasiTopN}")

Top-N recommendations for user-1: ['item-2', 'item-4', 'item-5']


In [ ]:
def getTopNRekomendasiV2(prediksiHybrid, ratingMatrix, user_index, N=5):
    pred_user = prediksiHybrid.iloc[user_index].values
    rating_user = ratingMatrix.iloc[user_index].values
    # Item yang belum dirating (nilai 0 atau NaN)
    if np.issubdtype(rating_user.dtype, np.number):
        belum_dirating_idx = np.where(rating_user == 0)[0]
    else:
        belum_dirating_idx = np.where(pd.isna(rating_user))[0]
    # Prediksi hanya untuk item yang belum dirating
    pred_belum_dirating = pred_user[belum_dirating_idx]
    # Urutkan dari skor tertinggi
    top_indices = pred_belum_dirating.argsort()[::-1][:N]
    # Ambil ID item asli
    top_item_ids = prediksiHybrid.columns.to_numpy()[belum_dirating_idx][top_indices]
    # Convert 'item-2' → 2 (kalau item ID masih dalam bentuk string)
    def parse_item_id(item):
        if isinstance(item, str) and item.startswith("item-"):
            return int(item.split("-")[1])
        return item  # sudah angka
    return [parse_item_id(i) for i in top_item_ids]

In [ ]:
cal_GetTopnRekomendasiV2 = getTopNRekomendasiV2(cal_HybridJaccard, dfToyData, 0, N=3)
cal_GetTopnRekomendasiV2

[2, 4, 5]

## NDCG

In [ ]:
# ndcg
groundTruth = [1, 2, 3, 4]
TopNrek = [1, 5, 2, 3, 15]
TopNrek = [1, 5, 2, 3, 15]

In [ ]:
def EvaluasiDCG(GroundTruth, TopNRekomendasi, N):
    # Inisialisasi skor DCG (Discounted Cumulative Gain) ke 0.0
    dcg_score = 0.0
    for i in range(min(N, len(TopNRekomendasi))):
        # Jika item rekomendasi ada dalam GroundTruth
        if TopNRekomendasi[i] in GroundTruth:
            # rumus DCG
            dcg_score += 1 / np.log2(i + 2)  
            print(f"Item {TopNRekomendasi[i]} ada di GroundTruth pada posisi {i+1}, DCG: {dcg_score}")
    #return nilai DCG
    return dcg_score

In [ ]:
cal_DCG = EvaluasiDCG(groundTruth, cal_GetTopnRekomendasiV2, 4)
print(f"Nilai DCG: {cal_DCG}")

Item 2 ada di GroundTruth pada posisi 1, DCG: 1.0
Item 4 ada di GroundTruth pada posisi 2, DCG: 1.6309297535714575
Nilai DCG: 1.6309297535714575


In [ ]:
def EvaluasiIDCG(N):
    # Inisialisasi IDCG
    idcg = 0.0
    # Hitung IDCG untuk N item teratas
    for n in range(1, N + 1):
        # Menghitung nilai relevansi ideal
        relevansi_ideal = 1  # Asumsikan rating tertinggi adalah 1
        # Menambahkan ke IDCG dengan formula relevansi/log2 posisi
        idcg += relevansi_ideal / math.log2(n + 1)
        print(f"Item ideal pada posisi {n}, IDCG: {idcg}")
    # Mengembalikan nilai IDCG
    return idcg

In [ ]:
cal_IDCG = EvaluasiIDCG(4)
print(f"Nilai IDCG: {cal_IDCG}")

Item ideal pada posisi 1, IDCG: 1.0
Item ideal pada posisi 2, IDCG: 1.6309297535714575
Item ideal pada posisi 3, IDCG: 2.1309297535714578
Item ideal pada posisi 4, IDCG: 2.5616063116448506
Nilai IDCG: 2.5616063116448506


In [ ]:
def EvaluasiNDCG(GroundTruth, TopNRekomendasi, N):
    # Hitung DCG
    dcg = EvaluasiDCG(GroundTruth, TopNRekomendasi, N)
    print(f"Nilai DCG: {dcg}")
    # Hitung IDCG
    idcg = EvaluasiIDCG(N)
    print(f"Nilai IDCG: {idcg}")
    # Menghitung NDCG
    rumusNDCG = dcg / idcg if idcg != 0 else 0

    return rumusNDCG


In [ ]:
calNDCG = EvaluasiNDCG(groundTruth, cal_GetTopnRekomendasiV2, 3)
print(f"Nilai NDCG: {calNDCG}")

Item 2 ada di GroundTruth pada posisi 1, DCG: 1.0
Item 4 ada di GroundTruth pada posisi 2, DCG: 1.6309297535714575
Nilai DCG: 1.6309297535714575
Item ideal pada posisi 1, IDCG: 1.0
Item ideal pada posisi 2, IDCG: 1.6309297535714575
Item ideal pada posisi 3, IDCG: 2.1309297535714578
Nilai IDCG: 2.1309297535714578
Nilai NDCG: 0.7653606369886217
